In [1]:
import gym
import matplotlib.pyplot as plt
from gym.wrappers.pixel_observation import PixelObservationWrapper
from gym import ObservationWrapper
from gym.spaces import Box
import numpy as np
import cv2

In [2]:
class GrayScaleObservation(ObservationWrapper):
    def __init__(self, env):
        super(GrayScaleObservation, self).__init__(env)

    def observation(self, observation):
        import cv2
        observation = cv2.cvtColor(observation['pixels'], cv2.COLOR_RGB2GRAY)
        return observation

In [3]:
class PixelDictWrapper(ObservationWrapper):

    def __init__(self, env, img_size:int, gaussian:bool=True):
        super().__init__(env)
        self.img_size = (img_size, img_size)
        self.gaussian = gaussian
        self.observation_space = Box(0, 255, shape=self.img_size, dtype=np.uint8)
        self.origin_obs = None
        self.origin_obs_dim = 3

    def observation(self, observation):
        pixel_obs = cv2.cvtColor(observation['pixels'], cv2.COLOR_RGB2GRAY, cv2.CV_32FC1)
        pixel_obs = cv2.resize(pixel_obs, self.img_size, interpolation=cv2.INTER_AREA)
        if self.gaussian:
            pixel_obs = cv2.GaussianBlur(pixel_obs, ksize=(3, 3), sigmaX=1., sigmaY=1., borderType=cv2.BORDER_DEFAULT)
        self.origin_obs = observation['state']
        return pixel_obs


def getenv_with_pixel_obs(env, img_size:int, gaussian:bool=True):
    return PixelDictWrapper(PixelObservationWrapper(env, pixels_only=False), img_size, gaussian)

In [6]:
# raw_env = RawPendulumEnv(g=9.81, biased=True, damping=1.0)
env = gym.make('Pendulum-v1', g=9.81)
env.reset()#如果不加这个reset会报错
# env = PixelObservationWrapper(env,pixels_only=False)#返回的是一个字典
# env = GrayScaleObservation(env)
# print(state)
# print(state['state'])
# print(state['pixels'])
# plt.imshow(state['pixels'])
imgsize = 16

env = getenv_with_pixel_obs(env, img_size=imgsize, gaussian=False)
obs = env.reset()

for i in range(3):
    u = env.action_space.sample()
    print(u)
    # print(obs.flatten()) #像素输出
    print(env.origin_obs) #像素输出对应的原始输出
    obs, _, _, _ = env.step(u)

[0.31320345]
[ 0.24384798  0.96981347 -0.91763395]
[-1.3620524]
[ 0.2514589   0.967868   -0.15711316]
[0.69432586]
[0.23445015 0.97212815 0.35068783]


In [5]:
import gym
from gym.wrappers import RescaleAction
base_env = gym.make('Pendulum-v1')
print(base_env.action_space)
wrapped_env = RescaleAction(base_env, min_action=0, max_action=1)
print(wrapped_env.action_space)

Box([-2.], [2.], (1,), float32)
Box([0.], [1.], (1,), float32)
